In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn import linear_model


! pip install optuna
import optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 7.7 MB/s 
     |████████████████████████████████| 209 kB 43.2 MB/s 
     |████████████████████████████████| 81 kB 9.3 MB/s 
     |████████████████████████████████| 78 kB 7.1 MB/s 
     |████████████████████████████████| 112 kB 37.8 MB/s 
     |████████████████████████████████| 147 kB 62.5 MB/s 
     |████████████████████████████████| 49 kB 5.8 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=885bfd8b9476a10efa6b1cf18ce96f1eee83fd5dc36d1ed17bce6771c225181b
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [2]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
#! chmod 600 /root/.kaggle/kaggle.json
! kaggle competitions download -c tabular-playground-series-aug-2022

from zipfile import ZipFile as unzipper
unzipped = unzipper('/content/tabular-playground-series-aug-2022.zip')
unzipped.extractall()
traindf=pd.read_csv('/content/train.csv')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  0% 0.00/2.27M [00:00<?, ?B/s]
100% 2.27M/2.27M [00:00<00:00, 140MB/s]


In [3]:
cdf=pd.read_csv('/content/preprocessed.csv')
cdf.drop(['id'],axis=1, inplace=True)
cdf.drop(['Unnamed: 0'],axis=1, inplace=True)
cdf['id']=cdf.index

In [4]:
train, test = cdf[cdf["ind"].eq("train")], cdf[cdf["ind"].eq("test")]

In [5]:
train=train.drop(['ind'],axis=1)
test=test.drop(['ind'],axis=1)

In [6]:
train['failure']=traindf['failure']

**Stratified K-fold:**

with LR

In [7]:
x=train.drop(['failure'], axis=1)
y=train['failure']

In [8]:
LR = linear_model.LogisticRegression()
  
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [28]:
accList = []
subpreds=[]
for train_index, test_index in skf.split(x, y):

    x_train_fold, x_test_fold = x.iloc[train_index], x.iloc[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

    model=LR.fit(x_train_fold, y_train_fold)

    subpreds=model.predict(test)

    accList.append(LR.score(x_test_fold, y_test_fold))

In [10]:
print("Accuracy of models: ",accList)

Accuracy of models:  [0.7873541588257433, 0.7873541588257433, 0.7873541588257433, 0.7871659766654121, 0.7869777945050809]


In [11]:
subpreds=LR.predict(test)

In [12]:
id=test['id']

In [30]:
submission=pd.DataFrame()
submission['failure']=subpreds
submission = submission.set_index(id.index)
submission['id']=id
submission=submission[['id', 'failure']]

In [31]:
submission.to_csv("submission.csv", index=False)